<a href="https://colab.research.google.com/github/Partha-SUST16/LLM_JailBreak_Low_Resource/blob/main/python_scripts/Generating_Common_Prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# prompt: sort each of the dataframes in dataframes.items(): by cosine_similarity in a descending order and keep only top 500

from google.colab import drive
import pandas as pd
import os
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

drive.mount('/content/drive')

# Define the folder path
folder_path = '/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/'

# List all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('translated_prompts_with_similarity.csv')]

# Sort the files to ensure consistent naming (optional)
csv_files.sort()

# Read each CSV file and store it in a dictionary of DataFrames
dataframes = {}
for i, file in enumerate(csv_files, start=1):
    df_name = f'df{i}'  # Create a name like df1, df2, etc.
    file_path = os.path.join(folder_path, file)
    dataframes[df_name] = pd.read_csv(file_path)

# Assuming 'cosine_similarity' is a column in your dataframes
for df_name, df in dataframes.items():
    if 'cosine_similarity' not in df.columns:
        print(f"Warning: 'cosine_similarity' column not found in {df_name}. Skipping sorting.")
        continue

    # Sort by cosine_similarity in descending order
    df_sorted = df.sort_values(by='cosine_similarity', ascending=False)

    # Keep only the top 500
    df_top_500 = df_sorted.head(1500)

    # Update the dataframe in the dictionary
    dataframes[df_name] = df_top_500

# Print the first few rows of each DataFrame to verify
for name, df in dataframes.items():
    print(f"{name}:")
    print(len(df))
    # print(df.head())

    print("\n")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
df1:
1500


df2:
1500


df3:
1500


df4:
1500


df5:
1500


df6:
1500


df7:
1500


df8:
1500


df9:
1500




In [ ]:
import pandas as pd
from functools import reduce

# Assume dataframes is a dictionary containing df1 to df9
dataframes_list = [dataframes[f'df{i}'] for i in range(1, 10)]

# Extract only the 'id' column from each DataFrame for merging
common_idx = reduce(
    lambda left, right: pd.merge(left, right, on='id', how='inner'),
    [df[['id']] for df in dataframes_list]
)['id']

# Filter, sort, and extract required information
filtered_dataframes = [df[df['id'].isin(common_idx)].sort_values(by='id') for df in dataframes_list]

# Compute lowest cosine similarity & unique language for each DataFrame
for i, df in enumerate(filtered_dataframes, start=1):
    # Find the lowest cosine similarity
    min_cosine_similarity = df['cosine_similarity'].min() if 'cosine_similarity' in df.columns else None

    # Extract unique language (since it's the same for all rows)
    language = df['language'].iloc[0] if 'language' in df.columns and not df['language'].empty else None

    print(f"Filtered and sorted df{i}: {len(df)} rows")
    print(f"Lowest Cosine Similarity: {min_cosine_similarity}")
    print(f"Language: {language}")
    # print(df.head(), "\n")

result_df = pd.DataFrame({'id': common_idx})

# Add cosine similarity values using only the language as column names
language_columns = []  # Store language column names for later processing

for i, df in enumerate(filtered_dataframes, start=1):
    # print("......................................................")
    language = df['language'].iloc[0] if 'language' in df.columns and not df['language'].empty else f"Language_{i}"
    # print(language)
    # if language == "Nepal (Nepali)":
    #     continue
    print(f"Adding cosine similarity values from df{i} with language: {language}")

    # Rename 'cosine_similarity' column to language name
    temp_df = df[['id', 'cosine_similarity']].rename(columns={'cosine_similarity': language})
    language_columns.append(language)  # Store the column name

    # Merge with the result dataframe
    result_df = result_df.merge(temp_df, on='id', how='left')

# Compute the minimum cosine similarity across all language columns for each row
result_df["min_cosine_similarity"] = result_df[language_columns].min(axis=1)
# Save the result to a CSV file
output_file = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/cosine_similarities.csv"
result_df.to_csv(output_file, index=False)

print(f"Cosine similarity data saved to: {output_file}")

Filtered and sorted df1: 446 rows
Lowest Cosine Similarity: 0.7439904005180051
Language: Bangladesh (Bengali)
Filtered and sorted df2: 446 rows
Lowest Cosine Similarity: 0.6066953947389107
Language: Maldives (Dhivehi)
Filtered and sorted df3: 446 rows
Lowest Cosine Similarity: 0.3525837972718274
Language: Bhutan (Dzongkha)
Filtered and sorted df4: 446 rows
Lowest Cosine Similarity: 0.765993333244503
Language: India (Hindi)
Filtered and sorted df5: 446 rows
Lowest Cosine Similarity: 0.2911939789365695
Language: Nepal (Nepali)
Filtered and sorted df6: 446 rows
Lowest Cosine Similarity: 0.5333842147468069
Language: Afghanistan (Pashto)
Filtered and sorted df7: 446 rows
Lowest Cosine Similarity: 0.3527072575323738
Language: Sri Lanka (Sinhala)
Filtered and sorted df8: 446 rows
Lowest Cosine Similarity: 0.6243086466500163
Language: Sri Lanka (Tamil)
Filtered and sorted df9: 446 rows
Lowest Cosine Similarity: 0.6842410366255928
Language: Pakistan (Urdu)
Adding cosine similarity values from d

In [ ]:
import pandas as pd

# File path
file_path = "/content/drive/MyDrive/UBC CS/CPSC 538P Project (Computer Security)/Translated Prompt V2/ne_translated_prompts_with_similarity.csv"

# Load the CSV
df = pd.read_csv(file_path)

# Check if 'cosine_similarity' column exists
if 'min_cosine_similarity' in result_df.columns:
    count_above_0_5 = (result_df['min_cosine_similarity'] > 0.8).sum()
    print(f"Number of rows with cosine_similarity: {count_above_0_5}")
else:
    print("Column 'cosine_similarity' not found in the dataset.")


Number of rows with cosine_similarity: 0


In [ ]:
import csv

def extract_prompts(fileA, fileB, output_file):
    # Step 1: Read the IDs from fileB into a set
    ids_in_fileB = set()
    with open(fileB, mode="r", encoding="utf-8") as bfile:
        reader = csv.DictReader(bfile)
        for row in reader:
            ids_in_fileB.add(row["id"])  # Assuming 'id' is the column name in fileB

    # Step 2: Read fileA and extract rows with matching IDs
    matching_rows = []
    with open(fileA, mode="r", encoding="utf-8") as afile:
        reader = csv.DictReader(afile)
        for row in reader:
            if row["id"] in ids_in_fileB:  # Check if the ID is in fileB
                matching_rows.append({
                    "id": row["id"],
                    "translated_prompt": row["translated_prompt"],
                    "cosine_similarity": row["cosine_similarity"]
                })

    # Step 3: Write the matching rows to the output file
    with open(output_file, mode="w", encoding="utf-8", newline="") as outfile:
        fieldnames = ["id", "translated_prompt", "cosine_similarity"]
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)

        # Write the header
        writer.writeheader()

        # Write the matching rows
        writer.writerows(matching_rows)

    print(f"Extracted {len(matching_rows)} prompts to {output_file}")

fileA = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/bn_translated_prompts_with_similarity.csv"  # Path to the prompt file (contains id, translated_prompt)
fileB = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/cosine_similarities.csv"  # Path to the file with IDs (contains id)
output_file = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/bn_common_prompts.csv"  # Path to save the extracted prompts

extract_prompts(fileA, fileB, output_file)

# Example usage
fileA = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/dv_translated_prompts_with_similarity.csv"  # Path to the prompt file (contains id, translated_prompt)
fileB = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/cosine_similarities.csv"  # Path to the file with IDs (contains id)
output_file = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/dv_common_prompts.csv"  # Path to save the extracted prompts

extract_prompts(fileA, fileB, output_file)

# Example usage
fileA = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/dz_translated_prompts_with_similarity.csv"  # Path to the prompt file (contains id, translated_prompt)
fileB = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/cosine_similarities.csv"  # Path to the file with IDs (contains id)
output_file = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/dz_common_prompts.csv"  # Path to save the extracted prompts

extract_prompts(fileA, fileB, output_file)

fileA = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/hi_translated_prompts_with_similarity.csv"  # Path to the prompt file (contains id, translated_prompt)
fileB = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/cosine_similarities.csv"  # Path to the file with IDs (contains id)
output_file = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/hi_common_prompts.csv"  # Path to save the extracted prompts

extract_prompts(fileA, fileB, output_file)

fileA = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/ne_translated_prompts_with_similarity.csv"  # Path to the prompt file (contains id, translated_prompt)
fileB = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/cosine_similarities.csv"  # Path to the file with IDs (contains id)
output_file = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/ne_common_prompts.csv"  # Path to save the extracted prompts

extract_prompts(fileA, fileB, output_file)

fileA = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/ps_translated_prompts_with_similarity.csv"  # Path to the prompt file (contains id, translated_prompt)
fileB = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/cosine_similarities.csv"  # Path to the file with IDs (contains id)
output_file = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/ps_common_prompts.csv"  # Path to save the extracted prompts

extract_prompts(fileA, fileB, output_file)

fileA = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/si_translated_prompts_with_similarity.csv"  # Path to the prompt file (contains id, translated_prompt)
fileB = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/cosine_similarities.csv"  # Path to the file with IDs (contains id)
output_file = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/si_common_prompts.csv"  # Path to save the extracted prompts

extract_prompts(fileA, fileB, output_file)

fileA = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/ta_translated_prompts_with_similarity.csv"  # Path to the prompt file (contains id, translated_prompt)
fileB = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/cosine_similarities.csv"  # Path to the file with IDs (contains id)
output_file = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/ta_common_prompts.csv"  # Path to save the extracted prompts

extract_prompts(fileA, fileB, output_file)

fileA = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/ur_translated_prompts_with_similarity.csv"  # Path to the prompt file (contains id, translated_prompt)
fileB = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/cosine_similarities.csv"  # Path to the file with IDs (contains id)
output_file = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/ur_common_prompts.csv"  # Path to save the extracted prompts

extract_prompts(fileA, fileB, output_file)

Extracted 446 prompts to /content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/bn_common_prompts.csv
Extracted 446 prompts to /content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/dv_common_prompts.csv
Extracted 446 prompts to /content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/dz_common_prompts.csv
Extracted 446 prompts to /content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/hi_common_prompts.csv
Extracted 446 prompts to /content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/ne_common_prompts.csv
Extracted 446 prompts to /content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/ps_common_prompts.csv
Extracted 446 prompts to /content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/si_common_prompts.csv
Extracted 446 prompts to /content/drive/M

In [ ]:
## Extract common english prompt

import csv

def extract_prompts(fileA, fileB, output_file):
    # Step 1: Read the IDs from fileB into a set
    ids_in_fileB = set()
    with open(fileB, mode="r", encoding="utf-8") as bfile:
        reader = csv.DictReader(bfile)
        for row in reader:
            ids_in_fileB.add(row["id"])  # Assuming 'id' is the column name in fileB

    # Step 2: Read fileA and extract rows with matching IDs
    matching_rows = []
    with open(fileA, mode="r", encoding="utf-8") as afile:
        reader = csv.DictReader(afile)
        for row in reader:
            if row["id"] in ids_in_fileB:  # Check if the ID is in fileB
                matching_rows.append({
                    "id": row["id"],
                    "original_prompt": row["original_prompt"]
                })

    # Step 3: Write the matching rows to the output file
    with open(output_file, mode="w", encoding="utf-8", newline="") as outfile:
        fieldnames = ["id", "original_prompt"]
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)

        # Write the header
        writer.writeheader()

        # Write the matching rows
        writer.writerows(matching_rows)

    print(f"Extracted {len(matching_rows)} prompts to {output_file}")

fileA = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/bn_translated_prompts_with_similarity.csv"  # Path to the prompt file (contains id, translated_prompt)
fileB = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/cosine_similarities.csv"  # Path to the file with IDs (contains id)
output_file = "/content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/en_common_prompts.csv"  # Path to save the extracted prompts

extract_prompts(fileA, fileB, output_file)

Extracted 446 prompts to /content/drive/MyDrive/CPSC 538P Project (Computer Security)/Translated Prompt V2/Commons/en_common_prompts.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
